In [2]:
import yfinance as yf
import pandas as pd
import os
import json
import numpy as np

# Fetch Data

In [3]:
chemin_csv = '/Users/gabry/Desktop/VS/portfolio-optimization/nasdaq_screener.csv'
chemin_json = '/Users/gabry/Desktop/VS/portfolio-optimization/tickers.json'

df = pd.read_csv(chemin_csv)

with open(chemin_json, 'r') as fichier:
    tickers = json.load(fichier)


# Convertir 'Market Cap' en numérique si ce n'est pas déjà le cas
df['Market Cap'] = pd.to_numeric(df['Market Cap'], errors='coerce')

# Filtrer les symboles avec un market cap > 1 000 000 000
filtered_df = df[df['Market Cap'] > 1000000000000]
tickers_serie = pd.Series(tickers)

# Filtrer les tickers
filtered_tickers = tickers_serie[tickers_serie.isin(filtered_df['Symbol'])]

print(filtered_tickers)
print(df.head())

20       AAPL
418      AMZN
3024     GOOG
3025    GOOGL
4494     MSFT
4864     NVDA
dtype: object
  Symbol                                               Name Last Sale  \
0      A             Agilent Technologies Inc. Common Stock   $111.78   
1     AA                    Alcoa Corporation Common Stock    $27.145   
2    AAC  Ares Acquisition Corporation Class A Ordinary ...   $10.745   
3   AACG   ATA Creativity Global American Depositary Shares     $1.10   
4   AACI            Armada Acquisition Corp. I Common Stock    $10.70   

   Net Change % Change    Market Cap        Country  IPO Year   Volume  \
0       1.520   1.379%  3.270541e+10  United States    1999.0   929920   
1      -0.505  -1.826%  4.844015e+09  United States    2016.0  3764116   
2       0.005   0.047%  7.586428e+08            NaN    2021.0     9824   
3      -0.030  -2.655%  3.478439e+07          China    2008.0     5762   
4       0.000    0.00%  0.000000e+00  United States    2021.0      100   

                  

In [45]:
indices = ['^GSPC', '^DJI', '^IXIC', '^FTSE', '^GDAXI', '^FCHI', '^N225', '^HSI', 'EEM', 'BND']
tickers = filtered_tickers

# Créez une liste vide pour stocker les données
tickers_list = []
benchmark_list = []

# Boucle pour récupérer les données pour chaque symbole
for ticker in tickers:
    # Utilisez la bibliothèque yfinance pour obtenir les données
    ticker = yf.Ticker(ticker)
    data = ticker.history(period="1y")  # Récupère les données pour la journée en cours

    # Ajoutez les données dans la liste
    tickers_list.append(data)

for indice in indices:
    # Utilisez la bibliothèque yfinance pour obtenir les données
    ticker = yf.Ticker(indice)
    data = ticker.history(period="1y")  # Récupère les données pour la journée en cours

    # Ajoutez les données dans la liste
    benchmark_list.append(data)

# Concaténez les données de chaque symbole en une seule DataFrame
stocks = pd.concat(tickers_list, keys=tickers, names=['Symbol'])
benchmark = pd.concat(benchmark_list, keys=indices, names=['Symbol'])

# Pivoter la DataFrame pour obtenir les prix de fermeture
# Assurez-vous que votre DataFrame a une colonne nommée 'Close' qui contient les prix de fermeture

benchmark_index_names = benchmark.index.names
benchmark_new_index = pd.MultiIndex.from_tuples([(idx[0], idx[1].replace(tzinfo=None)) for idx in benchmark.index], names=benchmark_index_names)
benchmark.index = benchmark_new_index
indices_price = benchmark.reset_index().pivot(index='Date', columns='Symbol', values='Close')

stocks_price = stocks.reset_index().pivot(index='Date', columns='Symbol', values='Close')
stocks_price.index = pd.to_datetime(stocks_price.index).date
stocks_price.index = pd.Index(stocks_price.index, name='Date')

BRK/A: No data found for this date range, symbol may be delisted
BRK/B: No data found for this date range, symbol may be delisted


# Data Pre Processing

In [46]:
# Obtenir les emplacements des valeurs manquantes
missing_values = indices_price.isna()

# Empiler les résultats pour obtenir les paires index-colonne
missing_locations = missing_values.stack()[missing_values.stack()]

# Afficher les emplacements des valeurs manquantes
print(missing_locations)

Date        Symbol
2022-12-01  BND       True
            EEM       True
            ^DJI      True
            ^GSPC     True
            ^IXIC     True
                      ... 
2023-11-23  EEM       True
            ^DJI      True
            ^GSPC     True
            ^IXIC     True
            ^N225     True
Length: 103, dtype: bool


In [34]:
#function to fill missing values of daily stock prices
#Mandatory requirements: (1) The dataset should have been cleaned of all empty rows 
#before missing values are filled, and 
#(2) the opening row of the dataset should not have any empty fields

def FillMissingValues(StockPrices):
    
    import numpy as np
    print('Fill missing values...')
    
    #identify positions of the missing values in StockPrices
    [rows, cols] = np.where(np.asarray(np.isnan(StockPrices)))
    
    #replace missing value with the previous day's price
    for t in range(rows.size):
        i=rows[t]
        j = cols[t]
        if (i-1) >= 0:           
            StockPrices.iloc[i,j]= StockPrices.iloc[i-1, j].copy()
        else:
            print('error')
    return StockPrices
    
#function to eliminate empty rows in a dataset
def EmptyRowsElimination(dfAssetPrices):

    # read dataset and extract its dimensions
    [Rows, Columns] = dfAssetPrices.shape
    dFrame = dfAssetPrices.iloc[0:Rows, 0:Columns]
    
    # call dropna method from Pandas 
    dFClean = dFrame.dropna(axis =0, how ='all')
    return dFClean

In [47]:
indices_price = EmptyRowsElimination(indices_price)
indices_price = FillMissingValues(indices_price)

Fill missing values...
error
error
error
error
error


In [72]:
# Assurez-vous que les index sont de type 'datetime' et ont le même format
indices_price.index = pd.to_datetime(indices_price.index)
stocks_price.index = pd.to_datetime(stocks_price.index)

# Trouver l'intersection des index
dates_communes = indices_price.index.intersection(stocks_price.index)

# Filtrer les deux DataFrames pour ne garder que les dates communes
indices_price = indices_price[indices_price.index.isin(dates_communes)]
stocks_price = stocks_price[stocks_price.index.isin(dates_communes)]


In [76]:
stocks_price

Symbol,AAPL,ABBV,ABT,ACN,ADBE,ADP,AMAT,AMD,AMGN,AMZN,...,TTE,TXN,UL,UNH,UNP,UPS,V,VZ,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2022-12-02,146.988403,157.305191,105.970078,295.436157,341.529999,264.221466,105.730011,74.980003,275.945801,94.129997,...,57.627472,172.254730,49.671265,528.207703,209.333023,182.146698,215.945160,35.567612,150.909698,106.232979
2022-12-05,145.814972,157.574310,103.342644,288.161713,334.089996,258.827423,105.452583,73.620003,275.365875,91.010002,...,56.828808,171.992950,49.179382,527.104309,205.755264,174.231461,211.996521,34.533562,149.363373,103.322357
2022-12-06,142.115646,157.362854,101.823036,280.946381,331.149994,253.090714,103.748383,70.269997,273.800140,88.250000,...,57.161587,169.035751,49.352982,531.320801,205.833252,168.403229,207.432770,34.365875,147.629913,100.450417
2022-12-07,140.156601,158.977615,102.754402,282.206360,326.679993,253.766174,105.135536,70.139999,276.187439,88.459999,...,56.334396,168.463715,49.015411,534.857544,206.886124,167.989655,206.172760,34.626717,146.861679,100.228020
2022-12-08,141.857086,159.544724,104.823021,287.580933,332.579987,255.812271,107.612564,70.470001,276.003784,90.349998,...,56.819298,170.994293,48.610325,539.793274,207.675766,172.759995,207.452606,34.561508,147.089066,100.972588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-27,189.789993,139.089996,102.709999,332.429993,619.270020,229.160004,150.809998,122.650002,264.269989,147.729996,...,68.699997,152.600006,47.830002,541.741882,219.850006,149.589996,254.139999,37.349998,156.770004,103.959999
2023-11-28,190.399994,138.080002,102.059998,332.559998,623.320007,228.990005,148.059998,122.010002,265.519989,147.029999,...,68.970001,152.899994,47.900002,538.692322,221.229996,150.740005,252.940002,37.470001,158.639999,103.900002
2023-11-29,189.369995,138.500000,103.629997,333.339996,617.390015,229.160004,149.360001,123.849998,266.600006,146.320007,...,67.860001,153.199997,47.529999,533.161133,221.470001,152.289993,254.229996,37.759998,156.080002,102.339996


In [77]:
# Enregistrer le DataFrame 'stocks_price' dans un fichier CSV
stocks_price.to_csv('/Users/gabry/Desktop/VS/portfolio-optimization/data/stocks_price.csv')

# Enregistrer le DataFrame 'indices_price' dans un fichier CSV
indices_price.to_csv('/Users/gabry/Desktop/VS/portfolio-optimization/data/indices_price.csv')